# Example: using OpenMM to run a simulation with a NequIP ML potential

You can run this example directly in your browser: [![Open On Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sef43/openmm-ml/blob/nequip/examples/run_nequip_pbc.ipynb)

## Install Conda

[Conda](https://docs.conda.io/) is a package and environment manager. On Google Colab, Conda is installed with [conda-colab](https://github.com/jaimergp/condacolab). On your computer, you should follow these [installation instructions](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html).

⚠️ Do not use conda-colab on your computer!

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_mambaforge() # use mamba on colab because it is faster than conda

## Install software

First install everything we can from [conda-forge](https://conda-forge.org/).
Then use pip.

⚠️ The installation might take up to 10 min!

In [ ]:
#https://github.com/openmm/openmm-torch/issues/88
%env CONDA_OVERRIDE_CUDA=11.2 
!mamba install -c conda-forge openmm-torch pytorch=1.12

!pip install torch-nl
!pip install nequip
!pip install git+https://github.com/sef43/openmm-ml@nequip

## Get the files we need to run the example

In [ ]:
!wget https://raw.githubusercontent.com/sef43/openmm-ml/nequip/example/minimal_toy_emt_model_deployed.pth

## Run simulation

In [2]:
import openmm
import openmm.app as app
import openmm.unit as unit
from openmmml import MLPotential
from sys import stdout



"""
Uses a deployed trained NequiP model, toy model with PBC:
nequip-train configs/minimal_toy_emt.yaml
nequip-deploy build --train-dir path/to/training/session/ deployed_model.pth
"""

### build the inital structure for the toy model
import ase.build
import ase.io
import numpy as np

rng = np.random.default_rng(123)
supercell = (4, 4, 4)
sigma = 0.1
base_atoms = ase.build.bulk('Cu', 'fcc', cubic=True).repeat(supercell)
print(base_atoms)
topology=openmm.app.Topology()
element = app.Element.getBySymbol('Cu')
chain = topology.addChain()
for particle in range(len(base_atoms)):
        residue = topology.addResidue('Cu', chain)
        topology.addAtom('Cu', element, residue)
cell=np.array(base_atoms.cell[:])*0.1 # A -> nm
topology.setPeriodicBoxVectors(cell)
positions=base_atoms.positions*0.1 # A->nm


# create a System with NequIP MLP

# need to specify the unit conversion factors from the NequIP model units to OpenMM units.
# distance: model is in Angstrom, OpenMM is in nanometers
A_to_nm = 0.1
# energy: model is in eV, OpenMM is in kJ/mol
eV_to_kJ_per_mol = 96.49

potential = MLPotential('nequip', model_path='minimal_toy_emt_model_deployed.pth', 
                        distance_to_nm=A_to_nm, 
                        energy_to_kJ_per_mol=eV_to_kJ_per_mol)

system = potential.createSystem(topology)


# run NPT
temperature=800.0*unit.kelvin
pressure=1.0*unit.bar
integrator = openmm.LangevinIntegrator(temperature, 1.0/unit.picoseconds, 5.0*unit.femtosecond)
simulation=app.Simulation(topology, system, integrator)
simulation.context.setPositions(positions)

system.addForce(openmm.MonteCarloBarostat(pressure, temperature))
simulation.context.reinitialize(preserveState=True)

simulation.reporters.append(app.PDBReporter('output.pdb', 100, enforcePeriodicBox=True))
simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True,
        potentialEnergy=True, temperature=True, volume=True))

simulation.step(1000)

# Minimize the energy
simulation.minimizeEnergy()
energy=simulation.context.getState(getEnergy=True).getPotentialEnergy()
print(energy)

Atoms(symbols='Cu256', pbc=True, cell=[14.44, 14.44, 14.44])
#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)"
100,-21447.6484375,342.13267980003405,3.012526006963853
200,-20752.3671875,491.1717814318535,3.012526006963853
300,-20442.591796875,620.3283807295974,3.0458274820734754
400,-20149.01171875,687.1150620095186,3.0939494791179367
500,-19926.572265625,742.1562597959239,3.0866775568197635
600,-19835.318359375,776.6294464035517,3.0913854556149896
700,-19711.41015625,770.2821841447494,3.1194645444818
800,-19555.7109375,775.7763669791223,3.140311457283665
900,-19544.390625,772.8282296350236,3.1010864402229994
1000,-19669.275390625,827.9967665462585,3.0992291634887996
-22125.158203125 kJ/mol
